In [29]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
import re
from sklearn.model_selection import train_test_split


In [10]:
with open('./data/raw_dataset.csv', 'r', encoding='utf-8') as f:
    data = pd.DataFrame([line.rstrip('\n') for line in f], columns=['text'])

In [17]:
def clean_string(text):
    # приведение к нижнему регистру
    text = text.lower()
    # удаление всего, кроме латинских букв, цифр и пробелов
    text = re.sub(r'[^a-z0-9\s]', '', text)
    # удаление дублирующихся пробелов, удаление пробелов по краям
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text
    


In [24]:
data['text'] = data['text'].apply(clean_string)

In [31]:
val_size = 0.2
train_texts, val_texts = train_test_split(data['text'], test_size=val_size, random_state=42)
val_texts, test_texts = train_test_split(val_texts, test_size=val_size, random_state=42)

print(f"Train texts: {len(train_texts)}, Val texts: {len(val_texts)}")


Train texts: 1280398, Val texts: 256080


In [32]:
class NextTokenDataset(Dataset):
    def __init__(self, texts: list, tokenizer, seq_len: int =7):
        self.samples = []
        texts = 
        pass

    def __len__(self):

        return len(self.samples)

    def __getitem__(self, idx: int):
        pass

In [35]:
d = NextTokenDataset(train_texts)

TypeError: NextTokenDataset.__init__() missing 1 required positional argument: 'tokenizer'